In [1]:
import pandas as pd
import numpy as np



### Get all the pillar names from the excel

In [2]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [3]:
col_names = ['Indicator','check', 'Data Source','Index','Filename','Sub-Pillar']

In [4]:
names = names[col_names]

In [5]:
names.head()

,Indicator,check,Data Source,Index,Filename,Sub-Pillar
0,Countries,NaN,United Nations,False,Countries,NaN
1,"Database of Global Administrative Areas (GADM,...",NaN,GADM maps and data,False,NaN,NaN
2,High Resolution Population Density Maps + Demo...,NaN,Facebook,False,NaN,NaN
3,population density vs openstreetmap object den...,NaN,Kontur,False,NaN,NaN
4,Population Density,Infrastructure,World Bank,False,population_density,Connectivity Technology


In [6]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [7]:
data_stats

,Filename,Indicator
check,,
Business,16,25
Foundations,8,13
Government,9,15
Infrastructure,39,48
People,34,47
Regulation,5,7
Strategy,1,1


### Foundations

In [8]:
bnames = names[(names.check=='Foundations')&(~names.Filename.isna())]#&(names.Index==False)]

In [9]:
bnames.head(25)

,Indicator,check,Data Source,Index,Filename,Sub-Pillar
148,Digital payments penetration,Foundations,Portulans Institute,TRUE,digital_payments_penetration,Digital Payments
149,% of population with digital finance account -...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
150,% of population with digital finance account -...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
154,% of population with ID (all),Foundations,World Bank,False,id4d_nid,Digital Legal Identity
155,% of population with ID (female),Foundations,World Bank,False,id4d_nid,Digital Legal Identity
157,can ID be used for transactions,Foundations,World Bank,False,id4d_services,Digital Legal Identity
158,Is personal data siloed,Foundations,World Bank,False,Egov_strategy,Digital Legal Identity
159,Open data index,Foundations,Open Knowledge Foundation,True,open_data_idx,Data Exchange


In [10]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()
subpillars = bnames['Sub-Pillar'].unique()

In [11]:
# get all file names
bfiles = bnames.Filename.unique()

In [12]:
bfiles

array(['digital_payments_penetration',
       'population_digital_financial_services', 'id4d_nid',
       'id4d_services', 'Egov_strategy', 'open_data_idx'], dtype=object)

In [13]:
subpillars

array(['Digital Payments', 'Digital Legal Identity', 'Data Exchange'],
      dtype=object)

In [14]:
# ls digital-readiness-assessment-main/processed/

In [15]:
##ict_goods and services not in process data

In [16]:
# formula for converting scale
def convert_rank(old_value, old_min=1, old_max=7, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

### 1. Digital payments penetration

In [17]:
indicators[0]
subpillars[0]

'Digital Payments'

In [18]:
# load data
indicator = indicators[0]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillar = subpillars[0]
print(subpillar)

Digital payments penetration
digital_payments_penetration
Digital Payments


In [19]:
df.head()

,RANK,COUNTRY/ECONOMY,VALUE,SCORE
0,1.0,Norway,0.85,100.00
1,2.0,Denmark,0.83,97.24
2,3.0,Finland,0.80,93.95
3,4.0,Sweden,0.80,93.08
4,5.0,Netherlands,0.76,89.01


In [20]:
# score looks like the one to use
df.describe()

,RANK,VALUE,SCORE
count,122.000000,122.000000,122.000000
mean,61.500000,0.315738,36.076967
std,35.362409,0.209012,24.867406
min,1.000000,0.010000,0.000000
25%,31.250000,0.150000,16.235000
50%,61.500000,0.270000,30.870000
75%,91.750000,0.440000,51.110000
max,122.000000,0.850000,100.000000


In [21]:
# df.Indicator.unique()

In [22]:
# create standard columns
df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['SCORE'] 
df['Sub-Pillar'] = subpillar
df['Year'] = 2017


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-100 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [23]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']].head(15)

In [24]:
# output scores to csv
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 2. % of population with digital finance account - registered


In [25]:
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[0]
subpillar = subpillars[0]
print(subpillar)

% of population with digital finance account - registered
population_digital_financial_services
Digital Payments


In [26]:
df.head(15)

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,28%,34%,36%,15%,23%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,38%,43%,44%,29%,34%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,40%,42%,53%,27%,38%,...,2%,6%,1%,1%,4%,0%,4%,2%,NaN,NaN
8,2011,ARB,Arab world,NaN,NaN,22%,30%,33%,11%,14%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,ARB,Arab world,NaN,NaN,30%,38%,42%,18%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df = df[(df.Year == 2017)]

# Must convert the data to float by removing the % sign
df['Account (% age 15+)'] = df['Account (% age 15+)'].str.replace('%','')
df['Account (% age 15+)'] = df['Account (% age 15+)'].astype(float)

In [28]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Account (% age 15+)'] 
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [29]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Afghanistan,2017,% of population with digital finance account -...,15.0,1.329670,True,Digital Payments
7,Albania,2017,% of population with digital finance account -...,40.0,2.703297,True,Digital Payments
10,Arab world,2017,% of population with digital finance account -...,37.0,2.538462,True,Digital Payments
13,United Arab Emirates,2017,% of population with digital finance account -...,88.0,5.340659,True,Digital Payments
16,Argentina,2017,% of population with digital finance account -...,49.0,3.197802,True,Digital Payments
...,...,...,...,...,...,...,...
479,World,2017,% of population with digital finance account -...,69.0,4.296703,True,Digital Payments
482,Kosovo,2017,% of population with digital finance account -...,52.0,3.362637,True,Digital Payments
487,South Africa,2017,% of population with digital finance account -...,69.0,4.296703,True,Digital Payments
490,Zambia,2017,% of population with digital finance account -...,46.0,3.032967,True,Digital Payments


In [30]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 3. % of population with digital finance account - active (90 days)

In [31]:
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[0]
subpillar = subpillars[0]
print(subpillar)

% of population with digital finance account - active (90 days)
population_digital_financial_services
Digital Payments


In [32]:
df.head()

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df = df[(df.Year == 2017)]

# Must convert the data to float by removing the % sign
df['Made or received digital payments in the past year (% age 15+)'] = df['Made or received digital payments in the past year (% age 15+)'].str.replace('%','')
df['Made or received digital payments in the past year (% age 15+)'] = df['Made or received digital payments in the past year (% age 15+)'].astype(float)

In [34]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Made or received digital payments in the past year (% age 15+)'] 
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [35]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Afghanistan,2017,% of population with digital finance account -...,11.0,1.217391,True,Digital Payments
7,Albania,2017,% of population with digital finance account -...,29.0,2.195652,True,Digital Payments
10,Arab world,2017,% of population with digital finance account -...,26.0,2.032609,True,Digital Payments
13,United Arab Emirates,2017,% of population with digital finance account -...,84.0,5.184783,True,Digital Payments
16,Argentina,2017,% of population with digital finance account -...,40.0,2.793478,True,Digital Payments
...,...,...,...,...,...,...,...
479,World,2017,% of population with digital finance account -...,52.0,3.445652,True,Digital Payments
482,Kosovo,2017,% of population with digital finance account -...,39.0,2.739130,True,Digital Payments
487,South Africa,2017,% of population with digital finance account -...,60.0,3.880435,True,Digital Payments
490,Zambia,2017,% of population with digital finance account -...,39.0,2.739130,True,Digital Payments


In [36]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator))

## 4. % of population with ID (Total)


In [37]:
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[1]
subpillar = subpillars[1]
print(subpillar)

% of population with ID (all)
id4d_nid
Digital Legal Identity


In [38]:
df.head(15)

,id,Economy,Country Code,Region,Income,OECD,Lending category,Other,GCC adjusted,Inclusion Criteria,...,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,1,Afghanistan,AFG,SAS,LIC,NaN,IDA,HIPC,NaN,INCLUDE,...,Voter,"13,549,892","7,296,096","36,373,176","18,191,591","9,336,665","8,854,905","18,181,585","9,398,623","8,782,976"
1,2,Albania,ALB,ECS,UMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Direct,"2,267,673","2,188,099","2,934,363","540,349","280,793","259,561","2,394,014","1,199,481","1,194,510"
2,3,Algeria,DZA,MEA,UMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"42,008,054","14,075,724","7,181,719","6,894,010","27,932,330","14,033,462","13,898,865"
3,4,Andorra,AND,ECS,HIC,NaN,..,NaN,NaN,EXCLUDE,...,Voter,NaN,NaN,"80,209","13,580","6,971","6,609","66,629","34,060","32,569"
4,5,Angola,AGO,SSF,LMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"30,774,205","16,389,360","8,157,462","8,231,869","14,384,845","6,937,053","7,447,820"
5,6,Antigua and Barbuda,ATG,LCN,HIC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"103,050","29,388","14,745","14,644","73,662","34,732","38,932"
6,7,Argentina,ARG,LCN,UMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"44,688,864","11,751,904","5,977,086","5,774,773","32,936,960","15,898,099","17,038,900"
7,8,Armenia,ARM,ECS,LMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"2,934,152","689,894","367,840","322,065","2,244,258","1,012,449","1,231,792"
8,9,Australia,AUS,EAS,HIC,OECD,..,NaN,NaN,EXCLUDE,...,Voter,"7,847,109","8,269,334","24,772,247","5,624,607","2,884,647","2,739,917","19,147,640","9,456,278","9,691,403"
9,10,Austria,AUT,ECS,HIC,OECD,..,EMU,NaN,EXCLUDE,...,Voter,"3,093,348","3,307,645","8,751,820","1,318,229","676,854","641,377","7,433,591","3,617,222","3,816,362"


In [39]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
# Subtract the unregister proportion column by 100 and absolute the value
df['data_col'] = df['UP in % of Country Population'].sub(100).abs()
df ['Year'] = 2018
df['Country Name'] = df.iloc[:,1]
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [40]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Afghanistan,2018,% of population with ID (all),67,3.857143,True,Digital Legal Identity
1,Albania,2018,% of population with ID (all),100,6.000000,True,Digital Legal Identity
2,Algeria,2018,% of population with ID (all),89,5.285714,True,Digital Legal Identity
3,Andorra,2018,% of population with ID (all),47,2.558442,True,Digital Legal Identity
4,Angola,2018,% of population with ID (all),44,2.363636,True,Digital Legal Identity
...,...,...,...,...,...,...,...
193,"Venezuela, RB",2018,% of population with ID (all),87,5.155844,True,Digital Legal Identity
194,Vietnam,2018,% of population with ID (all),96,5.740260,True,Digital Legal Identity
195,"Yemen, Rep.",2018,% of population with ID (all),50,2.753247,True,Digital Legal Identity
196,Zambia,2018,% of population with ID (all),44,2.363636,True,Digital Legal Identity


In [41]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

In [42]:
## 5. % of population with ID (Female)

In [43]:
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[1]
subpillar = subpillars[1]
print(subpillar)

% of population with ID (female)
id4d_nid
Digital Legal Identity


In [44]:
df.head(15)

,id,Economy,Country Code,Region,Income,OECD,Lending category,Other,GCC adjusted,Inclusion Criteria,...,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,1,Afghanistan,AFG,SAS,LIC,NaN,IDA,HIPC,NaN,INCLUDE,...,Voter,"13,549,892","7,296,096","36,373,176","18,191,591","9,336,665","8,854,905","18,181,585","9,398,623","8,782,976"
1,2,Albania,ALB,ECS,UMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Direct,"2,267,673","2,188,099","2,934,363","540,349","280,793","259,561","2,394,014","1,199,481","1,194,510"
2,3,Algeria,DZA,MEA,UMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"42,008,054","14,075,724","7,181,719","6,894,010","27,932,330","14,033,462","13,898,865"
3,4,Andorra,AND,ECS,HIC,NaN,..,NaN,NaN,EXCLUDE,...,Voter,NaN,NaN,"80,209","13,580","6,971","6,609","66,629","34,060","32,569"
4,5,Angola,AGO,SSF,LMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"30,774,205","16,389,360","8,157,462","8,231,869","14,384,845","6,937,053","7,447,820"
5,6,Antigua and Barbuda,ATG,LCN,HIC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"103,050","29,388","14,745","14,644","73,662","34,732","38,932"
6,7,Argentina,ARG,LCN,UMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"44,688,864","11,751,904","5,977,086","5,774,773","32,936,960","15,898,099","17,038,900"
7,8,Armenia,ARM,ECS,LMC,NaN,IBRD,NaN,NaN,INCLUDE,...,Voter,NaN,NaN,"2,934,152","689,894","367,840","322,065","2,244,258","1,012,449","1,231,792"
8,9,Australia,AUS,EAS,HIC,OECD,..,NaN,NaN,EXCLUDE,...,Voter,"7,847,109","8,269,334","24,772,247","5,624,607","2,884,647","2,739,917","19,147,640","9,456,278","9,691,403"
9,10,Austria,AUT,ECS,HIC,OECD,..,EMU,NaN,EXCLUDE,...,Voter,"3,093,348","3,307,645","8,751,820","1,318,229","676,854","641,377","7,433,591","3,617,222","3,816,362"


In [45]:
df['% of UP that is Female'] = df['% of UP that is Female'].replace('n/a',np.nan)

In [46]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
# Subtract the unregister proportion column by 100 and absolute the value
df['data_col'] = df['% of UP that is Female'].sub(100).abs()
df ['Year'] = 2018
df['Country Name'] = df.iloc[:,1]
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [47]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Afghanistan,2018,% of population with ID (female),45.0,3.25,True,Digital Legal Identity
1,Albania,2018,% of population with ID (female),28.0,2.40,True,Digital Legal Identity
2,Algeria,2018,% of population with ID (female),NaN,NaN,True,Digital Legal Identity
3,Andorra,2018,% of population with ID (female),NaN,NaN,True,Digital Legal Identity
4,Angola,2018,% of population with ID (female),NaN,NaN,True,Digital Legal Identity
...,...,...,...,...,...,...,...
193,"Venezuela, RB",2018,% of population with ID (female),NaN,NaN,True,Digital Legal Identity
194,Vietnam,2018,% of population with ID (female),NaN,NaN,True,Digital Legal Identity
195,"Yemen, Rep.",2018,% of population with ID (female),NaN,NaN,True,Digital Legal Identity
196,Zambia,2018,% of population with ID (female),NaN,NaN,True,Digital Legal Identity


In [48]:
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 6. can ID be used for transactions



In [49]:
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[1]
subpillar = subpillars[1]
print(subpillar)

can ID be used for transactions
id4d_services
Digital Legal Identity


In [50]:
df.head(20)

,ID,Economy,Region,Income,Code,Inclusion Criteria,National ID,NID Issued at Birth,Mandatory NID age,Digitized ID system,...,URL,NID card/number name,NID cost,Primary CR entity,CR Gov Entity URL,CR entity category,Mandatory Birth Registration Period,Birth Registration cost,data_country,data_year
0,1.0,Afghanistan,SAS,LIC,AFG,INCLUDE,1,1,0,1,...,http://mcit.gov.af/en/page/7081,Tazkira / eNID,Afs 30-35 (US$ 0.5),Ministry of Interior Affairs,"http://moi.gov.af/en/page/7180 , http://moi.go...",2.0,6 m,free,NaN,NaN
1,2.0,Albania,ECS,UMC,ALB,INCLUDE,1,0,16,1,...,http://www.punetebrendshme.gov.al,Letërnjoftimi / Albanian Identity Card,10 euros,"Vital Statistics Offices , Ministry of Interio...",http://www.punetebrendshme.gov.al/,5.0,60 d,100 Leke,NaN,NaN
2,3.0,Algeria,MEA,UMC,DZA,INCLUDE,1,0,18,1,...,http://www.interieur.gov.dz/,Carte Nationale d'Identité Biométrique Electro...,free,Ministry of the Interior and Local Governments,http://www.interieur.gov.dz,2.0,5 - 60 d,free,NaN,NaN
3,4.0,Andorra,ECS,HIC,AND,EXCLUDE,0,,-,-,...,None,-,-,Civil Registry Office,http://www.registrecivil.ad,1.0,15 d,free,NaN,NaN
4,5.0,Angola,SSF,LMC,AGO,INCLUDE,1,0,10,1,...,http://www.minjusdh.gov.ao/VerPrestadorServico...,Bilhete de Identidade (National ID card),15 kwanza,Direcção Nacional dos Registos e do Notariado ...,http://www.minjusdh.gov.ao,1.0,5 d,free,NaN,NaN
5,6.0,Antigua and Barbuda,LCN,HIC,ATG,INCLUDE,0,,-,-,...,None,-,-,"Civil Registry, Ministry of Justice and Legal ...",http://www.legalaffairs.gov.ag/,1.0,30 d,EC 10,NaN,NaN
6,7.0,Argentina,LCN,UMC,ARG,INCLUDE,1,1,0,1,...,http://www.nuevodni.gov.ar/inicio/index.php,DNI (Documento Nacional de Identidad) / SIBIOS...,free,"Regional Civil Registries, Provincial Authorit...",http://www.mininterior.gov.ar/renaper/renaper.php,6.0,40 d,free,NaN,NaN
7,8.0,Armenia,ECS,LMC,ARM,INCLUDE,1,1,16,1,...,https://www.ekeng.am/hy/,National ID Card / National Passport,free,"Civil Registry Office, RA Ministry of Justice",http://www.moj.am/services/civil_registry/item...,1.0,1 y,free,NaN,NaN
8,9.0,Australia,EAS,HIC,AUS,EXCLUDE,0,,-,-,...,None,-,-,"Registrar-General, Departments of Justice",http://www.australia.gov.au/topics/law-and-jus...,1.0,6 m,free,NaN,NaN
9,10.0,Austria,ECS,HIC,AUT,EXCLUDE,1,0,0,1,...,https://www.help.gv.at/Portal.Node/hlpd/public...,Identitätsausweis / Personalausweis (Austrian ...,61.5 EUR,"Register Office, Baby-Point",https://www.help.gv.at/Portal.Node/hlpd/public...,2.0,1 m,free,NaN,NaN


In [51]:
df['Digitized ID system']= df['Digitized ID system'].replace('-',np.nan)
df['Digitized ID system']= df['Digitized ID system'].astype(float)

In [52]:
df.head(15)

,ID,Economy,Region,Income,Code,Inclusion Criteria,National ID,NID Issued at Birth,Mandatory NID age,Digitized ID system,...,URL,NID card/number name,NID cost,Primary CR entity,CR Gov Entity URL,CR entity category,Mandatory Birth Registration Period,Birth Registration cost,data_country,data_year
0,1.0,Afghanistan,SAS,LIC,AFG,INCLUDE,1,1,0,1.0,...,http://mcit.gov.af/en/page/7081,Tazkira / eNID,Afs 30-35 (US$ 0.5),Ministry of Interior Affairs,"http://moi.gov.af/en/page/7180 , http://moi.go...",2.0,6 m,free,NaN,NaN
1,2.0,Albania,ECS,UMC,ALB,INCLUDE,1,0,16,1.0,...,http://www.punetebrendshme.gov.al,Letërnjoftimi / Albanian Identity Card,10 euros,"Vital Statistics Offices , Ministry of Interio...",http://www.punetebrendshme.gov.al/,5.0,60 d,100 Leke,NaN,NaN
2,3.0,Algeria,MEA,UMC,DZA,INCLUDE,1,0,18,1.0,...,http://www.interieur.gov.dz/,Carte Nationale d'Identité Biométrique Electro...,free,Ministry of the Interior and Local Governments,http://www.interieur.gov.dz,2.0,5 - 60 d,free,NaN,NaN
3,4.0,Andorra,ECS,HIC,AND,EXCLUDE,0,,-,NaN,...,None,-,-,Civil Registry Office,http://www.registrecivil.ad,1.0,15 d,free,NaN,NaN
4,5.0,Angola,SSF,LMC,AGO,INCLUDE,1,0,10,1.0,...,http://www.minjusdh.gov.ao/VerPrestadorServico...,Bilhete de Identidade (National ID card),15 kwanza,Direcção Nacional dos Registos e do Notariado ...,http://www.minjusdh.gov.ao,1.0,5 d,free,NaN,NaN
5,6.0,Antigua and Barbuda,LCN,HIC,ATG,INCLUDE,0,,-,NaN,...,None,-,-,"Civil Registry, Ministry of Justice and Legal ...",http://www.legalaffairs.gov.ag/,1.0,30 d,EC 10,NaN,NaN
6,7.0,Argentina,LCN,UMC,ARG,INCLUDE,1,1,0,1.0,...,http://www.nuevodni.gov.ar/inicio/index.php,DNI (Documento Nacional de Identidad) / SIBIOS...,free,"Regional Civil Registries, Provincial Authorit...",http://www.mininterior.gov.ar/renaper/renaper.php,6.0,40 d,free,NaN,NaN
7,8.0,Armenia,ECS,LMC,ARM,INCLUDE,1,1,16,1.0,...,https://www.ekeng.am/hy/,National ID Card / National Passport,free,"Civil Registry Office, RA Ministry of Justice",http://www.moj.am/services/civil_registry/item...,1.0,1 y,free,NaN,NaN
8,9.0,Australia,EAS,HIC,AUS,EXCLUDE,0,,-,NaN,...,None,-,-,"Registrar-General, Departments of Justice",http://www.australia.gov.au/topics/law-and-jus...,1.0,6 m,free,NaN,NaN
9,10.0,Austria,ECS,HIC,AUT,EXCLUDE,1,0,0,1.0,...,https://www.help.gv.at/Portal.Node/hlpd/public...,Identitätsausweis / Personalausweis (Austrian ...,61.5 EUR,"Register Office, Baby-Point",https://www.help.gv.at/Portal.Node/hlpd/public...,2.0,1 m,free,NaN,NaN


In [53]:
# create standard columns
df.rename(columns={'Economy':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Year'] = 2018
df['Indicator'] = indicator
df['data_col'] = df['Digitized ID system']
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [54]:
df = df[['Country Name','Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Afghanistan,2018,can ID be used for transactions,1.0,6.0,True,Digital Legal Identity
1,Albania,2018,can ID be used for transactions,1.0,6.0,True,Digital Legal Identity
2,Algeria,2018,can ID be used for transactions,1.0,6.0,True,Digital Legal Identity
3,Andorra,2018,can ID be used for transactions,NaN,NaN,True,Digital Legal Identity
4,Angola,2018,can ID be used for transactions,1.0,6.0,True,Digital Legal Identity
...,...,...,...,...,...,...,...
194,"Venezuela, RB",2018,can ID be used for transactions,1.0,6.0,True,Digital Legal Identity
195,Vietnam,2018,can ID be used for transactions,1.0,6.0,True,Digital Legal Identity
196,"Yemen, Rep.",2018,can ID be used for transactions,1.0,6.0,True,Digital Legal Identity
197,Zambia,2018,can ID be used for transactions,1.0,6.0,True,Digital Legal Identity


In [55]:
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 7. Is personal data siloed


In [56]:
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# Must limit the database to the first 206 rows, the remaining rows do not contain any useful information
df = df.iloc[0:205,:]

# Drop the superfluous rows by dropping na
df = df[df['#'].notna()]

# Must convert data in the DPL column into float
df['DPL'] = df['DPL'].astype(float, errors = 'ignore')

subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Is personal data siloed
Egov_strategy
Digital Legal Identity


In [57]:
df.head(104)

,#,Flag,Code,Cnum,Economy,Level,Population,GNI,GNIPC,e-Government,...,NGTI-1,NGTI-2,NGTI-3,NGTI-4,GTI-1,GTI-2,GTI-3,GTI-4,data_country,data_year
0,1,NaN,AFG,4.0,Afghanistan,LIC,"38,928","20,726",540,https://mcit.gov.af/node/6938,...,0.69,0.52,0.31,0.62,-0.09,-0.34,0.02,-0.12,NaN,NaN
1,2,NaN,ALB,8.0,Albania,UMIC,"2,878","14,949","5,240",https://e-albania.al/,...,0.81,0.78,0.60,0.77,0.29,0.62,0.74,0.26,NaN,NaN
2,3,NaN,DZA,12.0,Algeria,LMIC,"43,851","170,722","3,970",https://www.mpttn.gov.dz/ar/content/%D8%A7%D9%...,...,0.73,0.46,0.02,0.60,0.06,-0.56,-0.68,-0.15,NaN,NaN
3,4,NaN,ADO,20.0,Andorra,HIC,77,"3,154","40,886",http://www.govern.ad,...,0.63,0.60,0.05,0.13,-0.28,-0.06,-0.62,-1.32,NaN,NaN
4,5,NaN,AGO,24.0,Angola,LMIC,"32,866","97,005","3,050",http://www.governo.gov.ao,...,0.68,0.69,0.21,0.61,-0.12,0.27,-0.23,-0.13,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,98,NaN,LSO,426.0,Lesotho,LMIC,"2,142","2,901","1,360",http://www.gov.ls,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,99,NaN,LBR,430.0,Liberia,LIC,"5,058","2,852",580,http://www.emansion.gov.lr,...,0.69,0.42,0.13,0.50,-0.08,-0.69,-0.42,-0.39,NaN,NaN
105,100,NaN,LBY,434.0,Libya,UMIC,"6,871","51,757","7,640",http://www.pm.gov.ly,...,0.26,0.31,0.01,0.50,-1.49,-1.09,-0.71,-0.42,NaN,NaN
106,101,NaN,LIE,438.0,Liechtenstein,HIC,38,"4,160","116,430",http://regierung.li,...,0.30,0.48,0.07,0.87,-1.35,-0.49,-0.57,0.51,NaN,NaN


In [58]:
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df.iloc[:,205]
df['Country Name'] = df['Economy']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [59]:
df = df[['Country Name','Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Afghanistan,2020,Is personal data siloed,-0.04,1.0,True,Digital Legal Identity
1,Albania,2020,Is personal data siloed,0.02,6.0,True,Digital Legal Identity
2,Algeria,2020,Is personal data siloed,-0.04,1.0,True,Digital Legal Identity
3,Andorra,2020,Is personal data siloed,0.02,6.0,True,Digital Legal Identity
4,Angola,2020,Is personal data siloed,0.02,6.0,True,Digital Legal Identity
5,Antigua and Barbuda,2020,Is personal data siloed,0.02,6.0,True,Digital Legal Identity
6,Argentina,2020,Is personal data siloed,0.02,6.0,True,Digital Legal Identity
7,Armenia,2020,Is personal data siloed,0.02,6.0,True,Digital Legal Identity
8,Australia,2020,Is personal data siloed,0.02,6.0,True,Digital Legal Identity
9,Austria,2020,Is personal data siloed,NaN,NaN,True,Digital Legal Identity


In [60]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 8. Open data index

In [61]:
bnames

,Indicator,check,Data Source,Index,Filename,Sub-Pillar
148,Digital payments penetration,Foundations,Portulans Institute,TRUE,digital_payments_penetration,Digital Payments
149,% of population with digital finance account -...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
150,% of population with digital finance account -...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
154,% of population with ID (all),Foundations,World Bank,False,id4d_nid,Digital Legal Identity
155,% of population with ID (female),Foundations,World Bank,False,id4d_nid,Digital Legal Identity
157,can ID be used for transactions,Foundations,World Bank,False,id4d_services,Digital Legal Identity
158,Is personal data siloed,Foundations,World Bank,False,Egov_strategy,Digital Legal Identity
159,Open data index,Foundations,Open Knowledge Foundation,True,open_data_idx,Data Exchange


In [62]:
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Open data index
open_data_idx
Data Exchange


In [63]:
df.head(15)

,id,site,name,slug,region,continent,rank,score
0,tw,global,Taiwan,taiwan,NaN,NaN,1,90
1,au,global,Australia,australia,NaN,NaN,2,79
2,gb,global,Great Britain,united_kingdom,NaN,NaN,2,79
3,fr,global,France,france,NaN,NaN,4,70
4,fi,global,Finland,finland,NaN,NaN,5,69
5,ca,global,Canada,canada,NaN,NaN,5,69
6,no,global,Norway,norway,NaN,NaN,5,69
7,br,global,Brazil,brazil,NaN,NaN,8,68
8,nz,global,New Zealand,new_zealand,NaN,NaN,8,68
9,nir,global,Northern Ireland,northern_ireland,NaN,NaN,10,67


In [64]:
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['score'] 
df['Country Name'] = df['name']
df['Year'] = 2016
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [65]:
df = df[['Country Name','Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Taiwan,2016,Open data index,90,6.000000,True,Data Exchange
1,Australia,2016,Open data index,79,5.382022,True,Data Exchange
2,Great Britain,2016,Open data index,79,5.382022,True,Data Exchange
3,France,2016,Open data index,70,4.876404,True,Data Exchange
4,Finland,2016,Open data index,69,4.820225,True,Data Exchange
5,Canada,2016,Open data index,69,4.820225,True,Data Exchange
6,Norway,2016,Open data index,69,4.820225,True,Data Exchange
7,Brazil,2016,Open data index,68,4.764045,True,Data Exchange
8,New Zealand,2016,Open data index,68,4.764045,True,Data Exchange
9,Northern Ireland,2016,Open data index,67,4.707865,True,Data Exchange


In [66]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

### Score Aggregating

In [67]:
import os


In [68]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('foundations')]

In [69]:
scores

['foundations_% of population with digital finance account - active (90 days)_scores.csv',
 'foundations_% of population with digital finance account - registered_scores.csv',
 'foundations_% of population with ID (all)_scores.csv',
 'foundations_% of population with ID (female)_scores.csv',
 'foundations_% of population with ID_scores.csv',
 'foundations_can ID be used for transactions_scores.csv',
 'foundations_Digital payments penetration_scores.csv',
 'foundations_Is personal data siloed_scores.csv',
 'foundations_Open data index_scores.csv']

In [70]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [71]:
df

,Unnamed: 0,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,2.0,Afghanistan,2017.0,% of population with digital finance account -...,11.0,1.217391,True,Digital Payments
1,7.0,Albania,2017.0,% of population with digital finance account -...,29.0,2.195652,True,Digital Payments
2,10.0,Arab world,2017.0,% of population with digital finance account -...,26.0,2.032609,True,Digital Payments
3,13.0,United Arab Emirates,2017.0,% of population with digital finance account -...,84.0,5.184783,True,Digital Payments
4,16.0,Argentina,2017.0,% of population with digital finance account -...,40.0,2.793478,True,Digital Payments
...,...,...,...,...,...,...,...,...
89,NaN,Antigua and Barbuda,2016.0,Open data index,5.0,1.224719,True,Data Exchange
90,NaN,Malawi,2016.0,Open data index,5.0,1.224719,True,Data Exchange
91,NaN,Barbados,2016.0,Open data index,4.0,1.168539,True,Data Exchange
92,NaN,Saint Kitts and Nevis,2016.0,Open data index,4.0,1.168539,True,Data Exchange


In [72]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1434 entries, 0 to 1433
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        166 non-null    float64
 1   Country Name      1434 non-null   object 
 2   Year              1236 non-null   float64
 3   Indicator         1434 non-null   object 
 4   data_col          1238 non-null   float64
 5   new_rank_score    1434 non-null   float64
 6   higher_is_better  1434 non-null   bool   
 7   Sub-Pillar        1236 non-null   object 
dtypes: bool(1), float64(4), object(3)
memory usage: 79.9+ KB


In [74]:
df.head(15)

,Unnamed: 0,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,NaN,-,2020.0,Is personal data siloed,NaN,0.000000,True,Digital Legal Identity
1,NaN,0.03,2020.0,Is personal data siloed,NaN,0.000000,True,Digital Legal Identity
2,2.0,Afghanistan,2017.0,% of population with digital finance account -...,11.00,1.217391,True,Digital Payments
3,NaN,Afghanistan,NaN,% of population with ID,67.00,3.857143,True,NaN
4,NaN,Afghanistan,2018.0,% of population with ID (female),45.00,3.250000,True,Digital Legal Identity
5,NaN,Afghanistan,2020.0,Is personal data siloed,-0.04,1.000000,True,Digital Legal Identity
6,NaN,Afghanistan,2018.0,% of population with ID (all),67.00,3.857143,True,Digital Legal Identity
7,NaN,Afghanistan,2017.0,% of population with digital finance account -...,15.00,1.329670,True,Digital Payments
8,NaN,Afghanistan,2016.0,Open data index,12.00,1.617978,True,Data Exchange
9,NaN,Afghanistan,2018.0,can ID be used for transactions,1.00,6.000000,True,Digital Legal Identity


In [75]:
df.describe()

,Unnamed: 0,Year,data_col,new_rank_score
count,166.000000,1236.000000,1238.000000,1434.000000
mean,249.325301,2017.890777,47.190969,3.834716
std,140.305563,1.105106,38.203251,2.169208
min,2.000000,2016.000000,-0.040000,0.000000
25%,130.750000,2017.000000,1.000000,2.144440
50%,251.500000,2018.000000,47.500000,4.511489
75%,367.000000,2018.000000,85.000000,6.000000
max,493.000000,2020.000000,100.000000,6.000000


In [76]:
# checking country names
sorted(df['Country Name'].unique().tolist())

['-',
 '0.03',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Arab world',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Chinese Taipei',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'Denmark',
 'Developing',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Asia & Pacific',
 'East Asia & Pacific (excluding high income)',
 'Ecuador',
 'Egypt',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Eth

In [77]:
# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()


In [78]:
# checking country names
sorted(df['Country Name'].unique().tolist())

['-',
 '0.03',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Arab world',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Chinese Taipei',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'Denmark',
 'Developing',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Asia & Pacific',
 'East Asia & Pacific (excluding high income)',
 'Ecuador',
 'Egypt',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Eth

In [79]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [80]:
agg_df.columns = ['agg_score', 'count_source' ]

In [81]:
max_number_sources = agg_df.describe()['count_source']['max']

In [82]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [83]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [84]:
agg_df.head(25)

,agg_score,count_source,agg_score_wt
Country Name,,,
North Macedonia,6.000000,1,0.750000
Taiwan,6.000000,1,0.750000
Gambia,6.000000,1,0.750000
Chinese Taipei,6.000000,1,0.750000
United States of America,6.000000,1,0.750000
Bahamas,6.000000,1,0.750000
Egypt,6.000000,1,0.750000
High income: OECD,5.672420,2,1.418105
Euro area,5.672420,2,1.418105


In [85]:
agg_df.to_csv('../pillar_scores/foundation_scores_v0.csv')

In [86]:
### Score Aggregating by Subpillars

In [87]:
df.insert(0,'Pillar','Foundations')
df

,Pillar,Unnamed: 0,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Foundations,NaN,-,2020.0,Is personal data siloed,NaN,0.000000,True,Digital Legal Identity
1,Foundations,NaN,0.03,2020.0,Is personal data siloed,NaN,0.000000,True,Digital Legal Identity
2,Foundations,2.0,Afghanistan,2017.0,% of population with digital finance account -...,11.00,1.217391,True,Digital Payments
3,Foundations,NaN,Afghanistan,NaN,% of population with ID,67.00,3.857143,True,NaN
4,Foundations,NaN,Afghanistan,2018.0,% of population with ID (female),45.00,3.250000,True,Digital Legal Identity
...,...,...,...,...,...,...,...,...,...
1429,Foundations,NaN,Zimbabwe,2018.0,can ID be used for transactions,1.00,6.000000,True,Digital Legal Identity
1430,Foundations,NaN,Zimbabwe,2020.0,Is personal data siloed,0.02,6.000000,True,Digital Legal Identity
1431,Foundations,NaN,Zimbabwe,2018.0,% of population with ID (female),NaN,0.000000,True,Digital Legal Identity
1432,Foundations,NaN,Zimbabwe,2016.0,Open data index,13.00,1.674157,True,Data Exchange


In [88]:
sub_df = df.groupby(['Pillar','Sub-Pillar','Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [89]:
sub_df.columns = ['agg_score', 'count_source' ]

In [90]:
max_number_sources = sub_df.describe()['count_source']['max']

In [91]:
sub_df['agg_score_wt'] = sub_df['agg_score']*(sub_df['count_source']/max_number_sources)

In [92]:
sub_df.to_csv('../subpillar_score/foundations_scores_subpillar_v0.csv')